In [1]:
#importing required libraries
import pandas as pd
import itertools 
from bs4 import BeautifulSoup
import requests
from requests import get
import time
from random import seed
from random import random
from random import randint

In [2]:
# specify the url format of the rental  website
url = 'https://www.pararius.com/apartments/amsterdam/page-'

In [1]:
# initialize a list called houses
houses = []

# initialize variable count at 1
count = 1


# first while loop that will run 100 times 

while count <= 100:
    # initialize variable new_count at 0
    new_count = 0
    # if loop that specifies the first page separately (many websites have a first page url format different than other pages)
    if count == 1:
        first_page = 'https://www.pararius.com/apartments/amsterdam/page-1'
        # request the response
        response = get(first_page)
        # parse through the html 
        html_soup = BeautifulSoup(response.text, 'html.parser')
        # in the html of the page, find all the bins with <li> and class:
        house_data = html_soup.find_all('li', class_="search-list__item search-list__item--listing")
        # I like to print where the program is on the screen so we can follow its progress and where any errors happened
        print(first_page)
        
        # if the response was not empty (if something was actually scraped)
        if house_data != []:
            # add to the list houses
            houses.extend(house_data)
            # random wait times
            value = random()
            scaled_value = 1 + (value * (9 - 5))
            print(scaled_value)
            time.sleep(scaled_value)
    # pages other than the first
    elif count != 1:
    # collect four and wait random times 
        url = 'https://www.pararius.com/apartments/amsterdam/page-' + str(count)
        print(url)
        response = get(url)
        html_soup = BeautifulSoup(response.text, 'html.parser')
        print(response)
        house_data = html_soup.find_all('li', class_="search-list__item search-list__item--listing")

        if house_data != []:
            houses.extend(house_data)
            value = random()
            scaled_value = 1 + (value * (9 - 5))
            print(scaled_value)
            time.sleep(scaled_value)

        # if you get empty response, stop the loop
        else:
            print('empty')
            break
            

    count += 1

NameError: name 'get' is not defined

In [ ]:
len(houses)

In [ ]:

## DATA FORMATTING 
## initializing lists and variables
count = 0
house_price = []
rental_agency = []
location = []
city = []
bedrooms = []
surface = []
## how long we are running the while loop for 
n = int(len(houses)) - 1

while count <= n:
    # running the loop through each html bin we scraped
    num = houses[int(count)]
    
    # getting the price: make sure to test this code a few times by itself to understand exactly which parameters will work 
    price = num.find_all('span',{"class":"listing-search-item__price"})[0].text
    house_price.append(price)
    df_price = pd.DataFrame({'house_price':house_price})
    df_price['house_price'] = df_price['house_price'].str.replace("\D","")
    df_price['house_price'] = df_price['house_price'].str.replace("per month","")
    
    # if you have a variable that is not present on all ads the ads, you can use try and except to avoid stopping the loop
    try:
        agency = num.find_all('a', href=True)[2].text
    except IndexError:
        agency = 'none'
    rental_agency.append(agency)
    df_agency = pd.DataFrame({'rental_agency':rental_agency})
    
   #getting the postcode: make sure to test this code a few times by itself to understand exactly which parameters will work 
    postcode = num.find('div',{"class":"listing-search-item__location"}).text
    location.append(postcode)
    df_postcode = pd.DataFrame({'postcode':location})
    df_postcode['postcode'] = df_postcode['postcode'].str.replace("\nApartment\n ","")
    df_postcode['postcode'] = df_postcode['postcode'].str.replace("\n","")
    df_postcode['postcode'] = df_postcode['postcode'].str.replace("\s","")
    df_postcode['postcode'] = df_postcode['postcode'].str.replace("               ","")
    df_postcode['postcode'] = df_postcode['postcode'].str.replace("new","")
    df_postcode['postcode'] = df_postcode['postcode'].str[0:6]
    
    #getting the number of bedrooms: make sure to test this code a few times by itself to understand exactly which parameters will work 
    bedrooms_num = num.find_all('dd',{"class":"illustrated-features__description"})[1].text
    bedrooms.append(bedrooms_num)
    df_bedrooms = pd.DataFrame({'bedrooms':bedrooms})
    df_bedrooms['bedrooms'] = df_bedrooms['bedrooms'].str.replace("\D","")
    
    #getting the sq meter size: make sure to test this code a few times by itself to understand exactly which parameters will work 
    size = num.find_all('dd',{"class":"illustrated-features__description"})[0].text
    surface.append(size)
    df_surface = pd.DataFrame({'surface':surface})
    df_surface['surface'] = df_surface['surface'].str.replace("\D","")
    
    print(count)
    
    count += 1

# concat all the different dataframes created, culminating in dfa (completed dataframe)
result = pd.concat([df_price, df_agency], axis=1, sort=False)
result2 = pd.concat([result, df_postcode], axis=1, sort=False)
result3 = pd.concat([result2, df_bedrooms], axis=1, sort=False)
df = pd.concat([result3, df_surface], axis=1, sort=False)

In [ ]:
df.head(100)

In [ ]:
df.to_csv("rental_data_amsterdam.csv",index="false")